# Librerias

In [1]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import OrdinalEncoder

# Cargamos datos

In [2]:
df = pd.read_csv('../01_data/test.csv', index_col = 0)
df.head()

,carat,cut,color,clarity,depth,table,x,y,z
id,,,,,,,,,
0,0.32,Ideal,I,SI1,60.5,58.0,4.43,4.49,2.70
1,1.24,Premium,I,SI1,62.9,60.0,6.80,6.74,4.26
2,1.66,Premium,D,SI1,62.0,59.0,7.55,7.60,4.70
3,0.75,Premium,D,SI2,60.6,56.0,5.94,5.90,3.59
4,1.50,Fair,E,SI2,64.8,55.0,7.26,7.15,4.67


In [3]:
# Reemplazamos los valores '0' por NaN en las columnas 'x', 'y' y 'z'
#df[['x', 'y', 'z']] = df[['x', 'y', 'z']].replace(0, np.nan)

# Eliminamos las filas que contienen NaN en las columnas 'x', 'y' y 'z'
#df = df.dropna(subset=['x', 'y', 'z'])

In [4]:
#df.drop_duplicates(inplace=True)

In [5]:
def detectar_outliers(lista_columnas, dataframe): 
    
    dicc_indices = {} # creamos un diccionario donde almacenaremos índices de los outliers
    
    # iteramos por la lista de las columnas numéricas de nuestro dataframe
    for col in lista_columnas:
        
        #calculamos los cuartiles Q1 y Q3
        Q1 = np.nanpercentile(dataframe[col], 25)
        Q3 = np.nanpercentile(dataframe[col], 75)
        
        # calculamos el rango intercuartil
        IQR = Q3 - Q1
        
        # calculamos los límites
        outlier_step = 1.5 * IQR
        
        # filtramos nuestro dataframe para indentificar los outliers
        outliers_data = dataframe[(dataframe[col] < Q1 - outlier_step) | (dataframe[col] > Q3 + outlier_step)]
        
        
        if outliers_data.shape[0] > 0: # chequeamos si nuestro dataframe tiene alguna fila. 
        
            dicc_indices[col] = (list(outliers_data.index)) # si tiene fila es que hay outliers y por lo tanto lo añadimos a nuestro diccionario
        

    
    return dicc_indices

In [6]:
df_numericas = df.select_dtypes(include=np.number)
lista_columnas = df_numericas.columns
lista_columnas

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [7]:
# aplicamos la función y sacamos los índices de los outliers.

ind = detectar_outliers(lista_columnas, df)
print(ind)

{'carat': [22, 27, 34, 39, 47, 112, 146, 194, 229, 239, 292, 307, 317, 354, 357, 360, 366, 384, 401, 403, 501, 511, 539, 608, 614, 705, 758, 763, 767, 769, 784, 796, 935, 976, 1025, 1033, 1046, 1128, 1150, 1237, 1267, 1285, 1353, 1373, 1385, 1398, 1439, 1481, 1507, 1549, 1569, 1573, 1588, 1591, 1684, 1689, 1708, 1753, 1778, 1795, 1796, 1804, 1844, 1851, 1900, 1905, 1907, 1954, 2051, 2063, 2077, 2115, 2160, 2288, 2297, 2341, 2354, 2366, 2396, 2461, 2463, 2465, 2474, 2567, 2615, 2642, 2656, 2662, 2755, 2901, 2913, 2915, 2932, 2955, 2970, 3088, 3112, 3122, 3141, 3170, 3219, 3236, 3251, 3275, 3297, 3353, 3362, 3425, 3428, 3443, 3454, 3458, 3555, 3564, 3565, 3631, 3658, 3677, 3767, 3834, 3920, 3935, 3985, 3989, 4007, 4014, 4084, 4120, 4156, 4188, 4189, 4229, 4232, 4259, 4285, 4313, 4345, 4369, 4476, 4496, 4525, 4526, 4531, 4542, 4594, 4610, 4612, 4640, 4656, 4669, 4678, 4689, 4740, 4747, 4800, 4810, 4897, 5011, 5076, 5102, 5135, 5179, 5216, 5249, 5302, 5327, 5361, 5433, 5454, 5458, 5519, 55

In [8]:
# lo que vamos a hacer es iterar por el diccionario que nos devuelve la función 
for k, v in ind.items():
    media = df[k].mean() # calculamos la media para cada una de las columnas que tenemos en nuestro diccionario
    for i in v: # iteremos por la lista de valores para cada columna
        df.loc[i,k] = media # usando el loc, reemmplazamos los valores de los outliers por la media que hemos calculado al inicio.  

In [9]:
# Instancia del codificador
oe = OrdinalEncoder(categories=[['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']])

# Aplicar Ordinal-Encoding a la variable 'cut'
df[['cut_encoded']] = oe.fit_transform(df[['cut']])
df

,carat,cut,color,clarity,depth,table,x,y,z,cut_encoded
id,,,,,,,,,,
0,0.32,Ideal,I,SI1,60.500000,58.0,4.43,4.49,2.70,4.0
1,1.24,Premium,I,SI1,62.900000,60.0,6.80,6.74,4.26,3.0
2,1.66,Premium,D,SI1,62.000000,59.0,7.55,7.60,4.70,3.0
3,0.75,Premium,D,SI2,60.600000,56.0,5.94,5.90,3.59,3.0
4,1.50,Fair,E,SI2,61.758487,55.0,7.26,7.15,4.67,0.0
...,...,...,...,...,...,...,...,...,...,...
13480,1.10,Premium,G,SI1,59.600000,60.0,6.74,6.70,4.00,3.0
13481,0.90,Very Good,D,SI1,62.100000,60.0,6.14,6.20,3.83,2.0
13482,0.30,Ideal,F,VS2,62.100000,53.3,4.30,4.32,2.68,4.0


In [10]:
# Instancia del codificador
oe = OrdinalEncoder(categories=[['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']])

# Aplicar Ordinal-Encoding a la variable 'cut'
df[['cut_encoded']] = oe.fit_transform(df[['cut']])
df

,carat,cut,color,clarity,depth,table,x,y,z,cut_encoded
id,,,,,,,,,,
0,0.32,Ideal,I,SI1,60.500000,58.0,4.43,4.49,2.70,4.0
1,1.24,Premium,I,SI1,62.900000,60.0,6.80,6.74,4.26,3.0
2,1.66,Premium,D,SI1,62.000000,59.0,7.55,7.60,4.70,3.0
3,0.75,Premium,D,SI2,60.600000,56.0,5.94,5.90,3.59,3.0
4,1.50,Fair,E,SI2,61.758487,55.0,7.26,7.15,4.67,0.0
...,...,...,...,...,...,...,...,...,...,...
13480,1.10,Premium,G,SI1,59.600000,60.0,6.74,6.70,4.00,3.0
13481,0.90,Very Good,D,SI1,62.100000,60.0,6.14,6.20,3.83,2.0
13482,0.30,Ideal,F,VS2,62.100000,53.3,4.30,4.32,2.68,4.0


In [11]:
# Instancia del codificador
oe = OrdinalEncoder(categories=[['J', 'I', 'H', 'F', 'G', 'D', 'E']])

# Aplicar Ordinal-Encoding a la variable 'cut'
df[['color_encoded']] = oe.fit_transform(df[['color']])
df

,carat,cut,color,clarity,depth,table,x,y,z,cut_encoded,color_encoded
id,,,,,,,,,,,
0,0.32,Ideal,I,SI1,60.500000,58.0,4.43,4.49,2.70,4.0,1.0
1,1.24,Premium,I,SI1,62.900000,60.0,6.80,6.74,4.26,3.0,1.0
2,1.66,Premium,D,SI1,62.000000,59.0,7.55,7.60,4.70,3.0,5.0
3,0.75,Premium,D,SI2,60.600000,56.0,5.94,5.90,3.59,3.0,5.0
4,1.50,Fair,E,SI2,61.758487,55.0,7.26,7.15,4.67,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...
13480,1.10,Premium,G,SI1,59.600000,60.0,6.74,6.70,4.00,3.0,4.0
13481,0.90,Very Good,D,SI1,62.100000,60.0,6.14,6.20,3.83,2.0,5.0
13482,0.30,Ideal,F,VS2,62.100000,53.3,4.30,4.32,2.68,4.0,3.0


In [12]:
# Instancia del codificador
oe = OrdinalEncoder(categories=[['SI2', 'I1', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']])

# Aplicar Ordinal-Encoding a la variable 'cut'
df[['clarity_encoded']] = oe.fit_transform(df[['clarity']])
df

,carat,cut,color,clarity,depth,table,x,y,z,cut_encoded,color_encoded,clarity_encoded
id,,,,,,,,,,,,
0,0.32,Ideal,I,SI1,60.500000,58.0,4.43,4.49,2.70,4.0,1.0,2.0
1,1.24,Premium,I,SI1,62.900000,60.0,6.80,6.74,4.26,3.0,1.0,2.0
2,1.66,Premium,D,SI1,62.000000,59.0,7.55,7.60,4.70,3.0,5.0,2.0
3,0.75,Premium,D,SI2,60.600000,56.0,5.94,5.90,3.59,3.0,5.0,0.0
4,1.50,Fair,E,SI2,61.758487,55.0,7.26,7.15,4.67,0.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13480,1.10,Premium,G,SI1,59.600000,60.0,6.74,6.70,4.00,3.0,4.0,2.0
13481,0.90,Very Good,D,SI1,62.100000,60.0,6.14,6.20,3.83,2.0,5.0,2.0
13482,0.30,Ideal,F,VS2,62.100000,53.3,4.30,4.32,2.68,4.0,3.0,3.0


In [13]:
# guardamos el csv con los valores cambiados
df.to_csv("../01_data/test_nonull_nooutliers_encoding.csv")